在transfomer中，masked multi-head attention是多头注意力机制的一种变体，通常用于解码器的部分。
它的主要目的是生成序列时，确保模型只关注**当前位置和之前位置**，从而避免未来的信息泄露

在解码器中，模型通过生成的部分序列来预测下一个token。在训练阶段，模型能够看到目标序列的整个内容，但为了让模型在解码
过程中模拟生成的过程，需要对尚未生成的词进行屏蔽(masking)，避免模型在预测下一个词的时候用到未来的信息。

Masked MHA的作用是在计算注意力得分时候，使用一个掩码矩阵，将未生成的部分屏蔽起来，使得模型只关注当前和之前的词。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MaskedMultiHeadAttention(nn.Module):
    def __init__(self, embd_dim, head_nums):
        super().__init__()
        self.head_nums = head_nums
        self.head_dim = embd_dim // head_nums
        # 精简线性计算的过程
        self.qkv = nn.Linear(embd_dim, embd_dim*3)
        self.proj = nn.Linear(embd_dim, embd_dim)


    def forward(self, x, mask=None):
        batch_size, seq_len, embd_dim = x.shape
        qkv = self.qkv(x)
        # 维度转换
        qkv = qkv.reshape(batch_size, seq_length, 3, self.head_nums, self.head_dim)
        Q, K, V = qkv.permute(2, 0, 3, 1, 4).unbind(0)

        # Q, K, V = qkv.permute(2, 0, 3, 1, 4).chunk(3, dim=0)
        print(Q.shape)

        dim_k = K.shape[-1]

        # 计算注意力权重
        attn = F.sigmoid((Q @ K.transpose(2, 3) / self.head_dim ** 0.5), dim=-1)
        if mask is not None:
            attn = attn.masked_fill(mask==0, float('-inf'))
        output = attn @ V
        # 将每个头的输出进行拼接
        output = output.transpose(2, 3).reshape(batch_size, seq_len, -1)
        output = self.proj(output)

        return output
        


dummy_input = torch.randn(1, 196, 768)
head_nums = 8
seq_length = 196
embedding_dim = 768
# 构建一个上三角矩阵，将未来位置设置为false，其余位置设置为true(当前信息的位置和之前信息的位置)
mask = torch.triu(torch.ones(seq_length, seq_length), diagonal=1)
masked_attention = MaskedMultiHeadAttention(embd_dim=embedding_dim, head_nums=head_nums)
output = masked_attention(dummy_input)
print(output.shape)